# Introducción a HuggingFace

En este notebook vamos a ver algunas aplicaciones de procesamiento de lenguaje natural usando la librería [transformers](https://huggingface.co/docs/transformers/index) de [HuggingFace](https://huggingface.co).

Los ejemplos aquí presentandos están inspirados en el libro [Natural Language Processing with Transformers](https://www.oreilly.com/library/view/natural-language-processing/9781098136789/).

En primer lugar comenzamos instalando la librería.

In [1]:
!pip install transformers[sentencepiece] -Uq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 39.6 MB/s eta 0:00:00


La librería transformers proporciona una API en capas que permite interactuar con la librería a distintos niveles de abstracción. En este notebook usaremos las *pipelines*, que abstraen todos los pasos necesarios para convertir texto plano en un conjunto de predicciones a partir de un modelo.

In [2]:
from transformers import pipeline
import pandas as pd

Suponed que tenemos el siguiente texto.

In [3]:
text = """Estimado proveedor de Amazon, la semana pasada pedi el libro "NLP with transformer" mediante un evio expres y,
          desafortunadamente cuando abri el paquete descubri que no era el libro que esperaba. Ademas estaba arrugado y
          sucio. como consumidora, esto es un problema y solicito el reembolso de mi dinero. Espero su respuesta.
          Un saludo.
"""

## Clasificación de sentimientos

El mensaje anterior, puede ser el típico mensaje dejado en una página de valoraciones, y, por lo tanto, nos puede interesar saber si el feedback que estamos recibiendo es positivo o negativo. Esto se conoce como análisis de sentimientos, y se puede averiguar usando HuggingFace. Para ello vamos a usar un modelo del [Hub](https://huggingface.co/models).

In [4]:
classifier = pipeline(task='text-classification',model='pysentimiento/robertuito-sentiment-analysis')

Ahora podemos a partir de nuestro modelo generar un paredicción. Las predicciones son siempre un diccionario de Python por lo que podemos mostrarlas de manera sencilla con Pandas.

In [5]:
outputs = classifier(text)
pd.DataFrame(outputs)

,label,score
0,NEG,0.900516


In [6]:
classifier(text)

[{'label': 'NEG', 'score': 0.9005159735679626}]

In [7]:
classifier('Me ha encantado este producto')

[{'label': 'POS', 'score': 0.9748007655143738}]

In [8]:
classifier('Me ha parecido horrible el producto')

[{'label': 'NEG', 'score': 0.9860497117042542}]

En la salida anterior, podemos ver que el sentimiento de la valoración es negativo.

## Reconocimiento de entidades

Predecir el sentimiento de un mensaje de un cliente, puede ser un buen primer paso, pero en este contexto es habitual tener que averiguar a qué servicio o producto se está refiriendo el cliente.

En procesado de lenguaje natural, los objetos reales como productos, lugares, o personas se conocen como entidades nombradas (en inglés *named entities*) y su extracción se puede realizar mediante una técnica llamada *named entity recognition*.

Desde el punto de vista de la librería transformers mediante los pipelines el uso de modelos para esta tarea es análogo a la clasificación de sentimientos.

In [9]:
ner_taggert = pipeline(model="mrm8488/bert-spanish-cased-finetuned-ner",aggregation_strategy="simple")
outputs = ner_taggert(text)
pd.DataFrame(outputs)

Some weights of the model checkpoint at mrm8488/bert-spanish-cased-finetuned-ner were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


,entity_group,score,word,start,end
0,ORG,0.945085,Amazon,22,28
1,MISC,0.998827,""" NLP with transformer """,61,83


Podemos ver que se han detectado distintas entidades como organizaciones (Amazon, Barcelona, Real Madrid), personas (Jónathan) y también lugares (Alemania).

## Respondiendo preguntas

Ahora nos podemos preguntar si es posible responder preguntas sobre un texto. Esto es lo que se conoce como *question answering*. Para ello debemos proporcionar una pregunta y el contexto de dicha pregunta.

In [10]:
reader = pipeline(model="mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es")
question = "¿Qué quiere el cliente?"
outputs = reader(question=question,context=text)
pd.DataFrame([outputs])

Some weights of the model checkpoint at mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


,score,start,end,answer
0,0.705088,300,322,reembolso de mi dinero


Como podemos ver la respuesta que nos da, no es excesivamente buena. Vamos a ver cómo solucionar esto utilizando un modelo en inglés, para ello será necesario realizar una traducción de nuestro texto.

## Traducción

La traducción es una tarea muy habitual en nuestro día a día. Para ella podemos usar Google Translate o también podemos usar la librería de HuggingFace.

In [11]:
translator_es_en = pipeline(model="Helsinki-NLP/opus-mt-es-en")
outputs = translator_es_en(text,clean_up_tokenization_spaces=True,min_length=100)
translated_text = outputs[0]['translation_text']
print(translated_text)

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Dear Amazon Provider, last week I ordered the book "NLP with Transformer" by means of an evio express and, unfortunately when I opened the package I discovered that it was not the book I expected. Besides it was wrinkled and dirty. as a consumer, this is a problem and I request reimbursement of my money. I await your reply. A greeting. I am grateful to you for your reply. I am grateful to you for your help. I am grateful to you for your help.


Ahora podemos ir combinando pipelines para producir una respuesta más satisfactoria que la vista anteriormente.

In [12]:
outputs = translator_es_en(question,clean_up_tokenization_spaces=True)
translated_question = outputs[0]['translation_text']
print(translated_question)

What does the client want?


In [13]:
reader1 = pipeline("question-answering")
outputs1 = reader1(question=translated_question,context=translated_text)
pd.DataFrame([outputs1])

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


,score,start,end,answer
0,0.364085,279,304,reimbursement of my money


In [14]:
translator_en_es = pipeline(model="Helsinki-NLP/opus-mt-en-es")
outputs = translator_en_es(outputs1['answer'],clean_up_tokenization_spaces=True)
translated_answer = outputs[0]['translation_text']
print(translated_answer)

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


reembolso de mi dinero


## Resumenes

El objetivo de la tarea de *summarization* es a partir de un texto, generar una versión corta del mismo que mantenga la información relevante.

In [20]:
import torch
from transformers import BertTokenizerFast, EncoderDecoderModel

In [22]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
ckpt = 'mrm8488/bert2bert_shared-spanish-finetuned-summarization'
tokenizer = BertTokenizerFast.from_pretrained(ckpt)
model = EncoderDecoderModel.from_pretrained(ckpt).to(device)

def generate_summary(text):

   inputs = tokenizer([text], padding="max_length", truncation=True, max_length=512, return_tensors="pt")
   input_ids = inputs.input_ids.to(device)
   attention_mask = inputs.attention_mask.to(device)
   output = model.generate(input_ids, attention_mask=attention_mask)
   return tokenizer.decode(output[0], skip_special_tokens=True)

generate_summary(text)


The following encoder weights were not tied to the decoder ['bert/pooler']
The following encoder weights were not tied to the decoder ['bert/pooler']
The following encoder weights were not tied to the decoder ['bert/pooler']
The following encoder weights were not tied to the decoder ['bert/pooler']


'Estimado proveedor de Amazon, la semana pasada pedi el libro " NLP with transformer " mediante un evio expres y, desafortunadamente cuando abri el paquete descubri que no era el libro que esperaba'

## Generación de textos

La última aplicación que vamos a ver es la generación de textos. Esto lo podríamos utilizar para generar una respuesta automática al cliente.

In [23]:
generator = pipeline(model="DeepESP/gpt2-spanish-medium")
respuesta = "Estimada Rocio, siento oir que su pedido fue confundido"
prompt = text + "\n\nRespuesta del comercial:\n"+ respuesta
outputs = generator(prompt,max_length=200)
print(outputs[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Estimado proveedor de Amazon, la semana pasada pedi el libro "NLP with transformer" mediante un evio expres y,
          desafortunadamente cuando abri el paquete descubri que no era el libro que esperaba. Ademas estaba arrugado y
          sucio. como consumidora, esto es un problema y solicito el reembolso de mi dinero. Espero su respuesta.
          Un saludo.


Respuesta del comercial:
Estimada Rocio, siento oir que su pedido fue confundido, pero ¿qué tipo de mercancía desea? 

   Repudio: 

No tiene importancia. 

 § 4. 

Para la mayoría de los lectores, el hecho de que haya en nuestra cultura mucha gente se muestra como un motivo de su "salida de nitrato de hierro"

 La gente suele encontrar
